# Call Options and Market Making

In [1]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import numpy as np
import pandas as pd

import src.black_scholes as bs
import src.option_hedger as oh
from src.paths import geom_brownian_path
import src.colnames as n

In [2]:
# Choose fixed values for Maturity, Strike and RiskFree rate
T = 1
K = 100
R = 0

# Choose some values for \sigma in percent (input 20 for 20%)
sigmas_prc = [5, 20, 60]

## General Descriptions

### Implied Volatility $\sigma^i$



### Zoom on Delta ($\Delta$)

* $\Delta$ is the sensivity of the Call price to a small variation of the underlying token spot price $S$
* $\Delta$ depends on various parameters (token spot $\S$, token volatility $\sigma$, time to maturity, risk-free rate,...)
  * Usually, we only use "time subscript" to represent it's value: $\Delta_t$
* For a Call Option : 
  * $0 \leq \Delta_t \leq 1$
  * $\Delta$ increases with token spot price


In [3]:
sigmas = [s/100 for s in sigmas_prc]

def fun_delta(x, s):
    return bs.call_delta(s, K, T, R, x)

# generate delta curves
res = []
for x in range(int(K/10), 2 * K):
    res.append([x, *[fun_delta(x, s) for s in sigmas]])

deltas = pd.DataFrame(res)
deltas.columns = ["Spot", *sigmas_prc]
deltas.set_index( "Spot", inplace=True)

# display
title_template = "Delta curves depending on sigma [Fixed K={K}, r={R}, T={T}y]"
title_plot = title_template.format(K=K, R=R, T=T)
fig = px.line(deltas, title=title_plot).add_vline(x=K, line_width=2, line_dash="dash")
fig.show()

In [4]:
# generate delta/gamma curves
res = []
for x in range(int(K/2), int(1.6 * K)):
    for t in [0.05, 0.25, 1]:
        res.append([x, t, bs.call_delta(0.2, K, t, R, x), bs.call_gamma(0.2, K, t, R, x)])

rdf = pd.DataFrame(res)
rdf.columns = ["Splot", "Ttm", "Delta", "Gamma"]

In [5]:
px.line(rdf, x="Splot", y ="Delta", color="Ttm", title="Delta depending on time to maturity")

### Zoom on Gamma ($\Gamma$)

* $\Gamma$ is the sensivity of the $\Delta$ to a small variation of the underlying token spot price $S$
  * If *positive* : this is the amount of token you have to buy to adjust your $\Delta$ position when price goes *up*.
  * If *negative* : this is the amount of token you have to buy to adjust your $\Delta$ position when price goes *down*.
* $\Gamma$ depends on various parameters (token spot $\S$, token volatility $\sigma$, time to maturity, risk-free rate,...)
  * Usually, we only use "time subscript" to represent it's value: $\Gamma_t$
* For a Call Option : 
  * $\Gamma$ is positive


In [6]:
sigmas = [s/100 for s in sigmas_prc]

# generate gamma curves
res = []
for x in range(int(K/10), 2 * K):
    res.append([x, *[bs.call_gamma(s, K, T, R, x) for s in sigmas]])

gammas = pd.DataFrame(res)
gammas.columns = ["Spot", *sigmas_prc]
gammas.set_index( "Spot", inplace=True)

# display
title_plot = "Gamma curves depending on sigma [Fixed K={K}, r={R}, T={T}y]".format(K=K, R=R, T=T)
px.line(gammas, title=title_plot).add_vline(x=K, line_width=2, line_dash="dash").show()

In [7]:
px.line(rdf, x="Splot", y ="Gamma", color="Ttm", title="Gamma depending on time to maturity")

## Replication in Action

### Call Price, and Replication/Hedging Portfolio

Suppose a nice world where : 

1. Our token spot price $S_t$ behaves like a B&S model with a fixed volatility $\sigma$
2. We can trade long/short, without limitation and impact on the maket

Let's put in place a portfolio that will hold $\Delta_t$ amount of token, at each time step :

* We will simulate a token spot price trajectory (daily, until the maturity of the call option)
* We will build a *hedging portfolio*, called $V$, by :
  * adjusting our quantity of token to match $\Delta_t$, every day

We want to **see** that this portfolio is linked to the **theoretical** value of the call option.

In [15]:
# Let's choose our parameters

# --- for the call option
T = 1               # time to maturity of the call option, in years
K = 100             # strike of the call option

# --- for the token price B&S model
SIGMA = 0.10        # volatility 0.2 = 20%
S0 = 100            # initial token price
MU = 0.0            # token spot price drift, 0.05=5%

# --- for the market (can be left at 0)
R = 0               # risk free rate

# --- for the random generator :)
SEED = 201


In [26]:
path, dt = geom_brownian_path(S0, MU, SIGMA, int(T*365), 1, SEED)
# px.line(path).show()

# replicate_call(sigma, K, T, r, hedge_threshs, hedge_win, path, dt, store=False)
states = oh.replicate_call(SIGMA, K, T, R, hedge_threshs=0.5, hedge_win=1*dt, path=path, dt=dt, store=True)

sdf = pd.DataFrame(states)
sdf[n.Option] = sdf.apply(lambda x: bs.call_price(SIGMA, K, x[n.Ttm], R, x[n.Price]), axis=1)
sdf[n.OptionHedged] = sdf[n.Option] - sdf[n.Valo]
# px.line(sdf).show()

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

visibles = [n.Option, n.Valo, n.OptionHedged]

def add_line(col, second_axis):
    visi = True if col in visibles else "legendonly"
    fig.add_trace(
        go.Scatter(x=sdf.index, y=sdf[col], 
            name="{0} ({1})".format(col, "right" if second_axis else "left"), visible=visi), 
        secondary_y=second_axis)

# Add traces
fig.add_trace(go.Bar(x=sdf.index, y=sdf[n.Delta], name="{0} ({1})".format(n.Delta, "left"), visible="legendonly"), secondary_y=False)
[add_line(col, True) for col in [n.Price, n.Option, n.Valo, n.OptionHedged]]
# [add_line(col, True) for col in [n.Delta]]

fig.update_layout(title_text="Option hedging", yaxis=dict(side="right"), yaxis2=dict(side="left"))
fig.show()

**Legend explanations**

* *Price*: token spot price over the days ($S_t$)
* *Option*: theoretical value of the Call option $C_t$ (B&S formula, with $\sigma$, etc...)
* *Repli*: value of our hedging portfolio $V_t$ over the days
* *OptionHedged*: Option - Repli
* *Delta*: quantity of token hold by $V_t$ every day (built to be the theoretical B&S $\Delta_t$)

**Observations**

1. At step 1 (or time 0), Call price $C_0$ comes from a probabilistic computation => it does not depend on the (randomly generated) path
2. At step 365 (= time 1), Call price is not model dependent anymore, it's value is $C_T=max(S_T - K, 0)$.
3. We observe that the spread $C_t - V_t$ is constant over the full time window, equals to $C_0$
4. We feel that the hedging portfolio $V$ only miss this $C_0$ amount of cash to perfectly replicate the call option at maturity (independently from the path)
   * Feel free to modify MU or to change the seed to see that the spread has always the same value 

#### Conclusion

**In a simulated market, the call price $C_0$ is the initial cost of it's hedging/replicating portfolio $V_t$.**


### Replication Risks

SPX illustrer un gain et une perte de pnl sur la couverture d un call

Ecrire la formule du l'ecart de couverture

Montrer que le portefeuille de couverture d'un call est une stratégie de MM (delta décroissant, gamma négatif)
M (cash; toke; option) valo start à 0 en 0